# Input & Label

In [1]:
from rtmag.paper.load import load_input_label, MyModel

In [2]:
input_files, label_files = load_input_label("/mnt/f/isee_dataset/11158")
len(input_files), len(label_files)

(66, 66)

In [3]:
idx = 13
input_files[idx]

PosixPath('/mnt/f/isee_dataset/11158/input/input_11158_20110214_000000.npz')

# Model prediction & Label (NLFFF, Potential)

In [4]:
from time import time

In [5]:
# mm = MyModel("/home/mgjeon/space/workspace/base/pino_CC_Hnorm_Unit_2/best_model.pt")
mm = MyModel("/home/mgjeon/space/workspace/base/uno_pi_cc_hnorm_unit_aug/best_model.pt")
# mm2 = MyModel("/home/mgjeon/space/workspace/base/uno_pi_cc_hnorm_unit_aug/model_60.pt", clip=6000)

Model loaded from epoch 40


In [6]:
start_time = time()
b = mm.get_pred(input_files[idx])
print(f"{time() - start_time} seconds")

0.7069485187530518 seconds


In [7]:
start_time = time()
B = mm.get_label(label_files[idx])
print(f"{time() - start_time} seconds")

1.3676214218139648 seconds


In [8]:
start_time = time()
Bp = mm.get_pot(label_files[idx])
print(f"{time() - start_time} seconds")

1.3119933605194092 seconds


In [9]:
start_time = time()
dx, dy, dz, dV = mm.get_dV(input_files[idx])
print(f"{time() - start_time} seconds")

0.004036664962768555 seconds


# Quantitative Evaluation

In [10]:
import rtmag.paper.metric as m

In [11]:
start_time = time()
res = m.evaluate(b, B)
print(f"{time() - start_time} seconds")

C_vec     : 0.96
C_cs      : 0.94
E_n'      : 0.70
E_m'      : 0.55
eps       : 0.91
1.2879064083099365 seconds


In [12]:
start_time = time()
res = m.evaluate_energy(b, B, Bp, dV, isprint=True)
print(f"{time() - start_time} seconds")

pred_E_1e33: 0.78
ref_E_1e33: 0.85
pot_E_1e33: 0.71
dV_1e23   : 1.53
pred_E_unit: 127841767447.58
ref_E_unit: 139871515001.88
pot_E_unit: 116420454927.01
C_vec     : 0.96
C_cs      : 0.94
E_n_prime : 0.70
E_m_prime : 0.55
eps       : 0.91
rel_l2_err: 0.28
2.154616117477417 seconds


In [14]:
start_time = time()
res = m.evaluate_energy(b[:, :, :128, :], B[:, :, :128, :], Bp[:, :, :128, :], dV, isprint=True)
print(f"{time() - start_time} seconds")

pred_E_1e33: 0.77
ref_E_1e33: 0.84
pot_E_1e33: 0.70
dV_1e23   : 1.53
pred_E_unit: 126722599735.84
ref_E_unit: 139057338988.71
pot_E_unit: 115651620541.42
C_vec     : 0.96
C_cs      : 0.93
E_n_prime : 0.70
E_m_prime : 0.62
eps       : 0.91
rel_l2_err: 0.28
1.119044303894043 seconds


In [15]:
start_time = time()
res = m.evaluate_energy(b[:, :, :64, :], B[:, :, :64, :], Bp[:, :, :64, :], dV, isprint=True)
print(f"{time() - start_time} seconds")

pred_E_1e33: 0.74
ref_E_1e33: 0.82
pot_E_1e33: 0.68
dV_1e23   : 1.53
pred_E_unit: 122278106122.15
ref_E_unit: 135255310580.55
pot_E_unit: 112190644904.81
C_vec     : 0.96
C_cs      : 0.88
E_n_prime : 0.68
E_m_prime : 0.46
eps       : 0.90
rel_l2_err: 0.28
0.5737972259521484 seconds


In [18]:
(dz/1e8) * 64

34.3470344543457

In [14]:
start_time = time()
res = m.evaluate(b[:, :, :64, :], B[:, :, :64, :])
print(f"{time() - start_time} seconds")

C_vec     : 0.96
C_cs      : 0.88
E_n'      : 0.68
E_m'      : 0.46
eps       : 0.90
0.27683591842651367 seconds


In [15]:
start_time = time()
res = m.evaluate(b[128:-128, 64:-64, :128, :], B[128:-128, 64:-64, :128, :])
print(f"{time() - start_time} seconds")

C_vec     : 0.97
C_cs      : 0.96
E_n'      : 0.75
E_m'      : 0.71
eps       : 0.89
0.16946101188659668 seconds


In [16]:
m.metrics_j(b)

(0.798784094055181, 80.43580965604433, 38.13338596568818)

In [17]:
m.metrics_j(B)

(0.685718401941427, 72.86644558109357, 12.196899046200091)

In [18]:
m.metrics_j(Bp)

(0.8039919919220176, 1.502768025184216, 1.5461015324053273)

In [19]:
start_time = time()
bf = mm.fine_tuning(dx=dx/1e8, dy=dy/1e8, dz=dz/1e8, 
                    w_reg=0.01, w_bc=10, w_ff=1, w_div=1,
                    lr=1e-5, n_epoch=10)
print(f"{time() - start_time} seconds")

0
1
2
3
4
5
6
7
8
9
8.491246938705444 seconds


In [20]:
m.metrics_j(bf)

(0.8021734432552533, 76.15129386857882, 37.754993212530664)

In [21]:
start_time = time()
res = m.evaluate(bf, B)
print(f"{time() - start_time} seconds")

C_vec     : 0.96
C_cs      : 0.94
E_n'      : 0.69
E_m'      : 0.52
eps       : 0.90
1.1230638027191162 seconds
